In [2]:
!pip install --ignore-installed --upgrade kaggle
!pip install --upgrade tensorflow-gpu==2.3.1
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c cassava-leaf-disease-classification
!unzip /content/cassava-leaf-disease-classification.zip

Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/67/b8/ba/041548f30a6fc058c9b3f79a5b7b6aea925a15dd1e5c4992a4/python_slugify-4.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/a0/5f06e1e1d463903cf0c0eebeb751791119ed7a4

Requirement already up-to-date: tensorflow-gpu==2.3.1 in /usr/local/lib/python3.6/dist-packages (2.3.1)


Streaming output truncated to the last 5000 lines.
  inflating: train_images/3954399974.jpg  
  inflating: train_images/3954487465.jpg  
  inflating: train_images/3954910918.jpg  
  inflating: train_images/3955391972.jpg  
  inflating: train_images/3955442838.jpg  
  inflating: train_images/3955739563.jpg  
  inflating: train_images/3955931830.jpg  
  inflating: train_images/3955972139.jpg  
  inflating: train_images/3956075690.jpg  
  inflating: train_images/3956077728.jpg  
  inflating: train_images/3956155774.jpg  
  inflating: train_images/3956271103.jpg  
  inflating: train_images/3956372146.jpg  
  inflating: train_images/3956407201.jpg  
  inflating: train_images/3956550570.jpg  
  inflating: train_images/3956605397.jpg  
  inflating: train_images/3956778160.jpg  
  inflating: train_images/3957562076.jpg  
  inflating: train_images/3957612771.jpg  
  inflating: train_images/3957807023.jpg  
  inflating: train_images/3957819631.jpg  
  inflating: train_images/395820132.jpg  
  in

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime
import os

tf.keras.regularizers.l2(l2=0.01)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

datagen = ImageDataGenerator(validation_split=0.2,
                             dtype=tf.float32, horizontal_flip=True, rotation_range=0.2)
train_csv = pd.read_csv(r"/content/train.csv")
train_csv["label"] = train_csv["label"].astype(str)
base_model = tf.keras.applications.EfficientNetB6(include_top=False, weights="imagenet")

model = tf.keras.Sequential([
	tf.keras.layers.Input((512, 512, 3)),
	tf.keras.layers.BatchNormalization(renorm=True),
	base_model,
	tf.keras.layers.LeakyReLU(),

	BatchNormalization(),

	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(256),

	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),

	tf.keras.layers.Dense(128),
	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),

	tf.keras.layers.Dropout(0.3),
	tf.keras.layers.LeakyReLU(),

	BatchNormalization(),

	tf.keras.layers.Dense(64),
	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),
	tf.keras.layers.Dense(32),
	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),
	tf.keras.layers.Dropout(0.3),

	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),

	tf.keras.layers.Dense(16),
	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),
	tf.keras.layers.Dense(8),
	tf.keras.layers.LeakyReLU(),
	BatchNormalization(),

	tf.keras.layers.Dense(5, activation='softmax')
])

loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2)
model.compile(
	optimizer=tf.keras.optimizers.SGD(0.04),
	loss='categorical_crossentropy',
	metrics=['categorical_accuracy'])

early = EarlyStopping(monitor='val_loss',
                      mode='min',
                      patience=5)
checkpoint_filepath = r"/content/temp/"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
	filepath=checkpoint_filepath,
	save_weights_only=True,
	monitor='val_categorical_accuracy',
	mode='max',
	save_best_only=True)
model.fit(datagen.flow_from_dataframe(dataframe=train_csv,
                                      directory=r"/content/train_images", x_col="image_id",
                                      y_col="label", target_size=(512, 512), class_mode="categorical", batch_size=8,
                                      subset="training", shuffle=True),
          callbacks=[early, model_checkpoint_callback, tensorboard_callback],
          epochs=10, validation_data=datagen.flow_from_dataframe(dataframe=train_csv,
                                                                 directory=r"/content/train_images",
                                                                 x_col="image_id",
                                                                 y_col="label", target_size=(512, 512),
                                                                 class_mode="categorical", batch_size=8,
                                                                 subset="validation", shuffle=True))
model.load_weights(checkpoint_filepath)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0
Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.
Epoch 1/10
   1/2140 [..............................] - ETA: 0s - loss: 1.5518 - categorical_accuracy: 0.3750WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  55/2140 [..............................] - ETA: 20:04 - loss: 1.5732 - categorical_accuracy: 0.3364

In [ ]:
model.save(r"/content/86effnetb6.h5",include_optimizer=False)

In [ ]:
model.evaluate(datagen.flow_from_dataframe(dataframe=train_csv,
                                                                directory=r"/content/train_images",
                                                                x_col="image_id",
                                                                y_col="label", target_size=(512, 512),
                                                                class_mode="categorical", batch_size=8,
                                                                subset="validation", shuffle=True))

Found 4279 validated image filenames belonging to 5 classes.
535/535 [==============================] - 56s 105ms/step - loss: 0.4721 - categorical_accuracy: 0.8645


[0.47205039858818054, 0.8644543290138245]

In [ ]:
import tensorflow as tf

print(tf.__version__)

2.3.1


In [ ]:
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash
!gcloud init


In [ ]:
!nvidia-smi

Mon Jan  4 03:07:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!gsutil iam ch allUsers:objectViewer gs://mithil

In [ ]:
!gsutil cp /content/86effnetb6.h5 gs://mithil/models

Copying file:///content/86effnetb6.h5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/733.5 MiB.                                    
